<a href="https://colab.research.google.com/github/navaneethsdk/Data-Driven-Astronomy/blob/master/Psycopg2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install postgresql server
!sudo apt-get -y -qq update
!sudo apt-get -y -qq install postgresql
!sudo service postgresql start

# Setup a password `postgres` for username `postgres`
!sudo -u postgres psql -U postgres -c "ALTER USER postgres PASSWORD 'postgres';"

# Setup a database with name `db` to be used
!sudo -u postgres psql -U postgres -c 'DROP DATABASE IF EXISTS db;'
!sudo -u postgres psql -U postgres -c 'CREATE DATABASE db;'

 * Starting PostgreSQL 10 database server
   ...done.
ALTER ROLE
ERROR:  database "db" is being accessed by other users
DETAIL:  There is 1 other session using the database.
ERROR:  database "db" already exists


An advantage of using a popular language is the high probability that programmers before us have found solutions to our problems already. As such, there exists a Python module which allows us to interface with a PostgreSQL database, called Psycopg2.

Using this module requires a quite advanced understanding of Python and SQL, so we'll start off with a very simple query and work through it step by step.

Let's suppose we want to implement the following SQL query:
SELECT 2 + 3;

In [ ]:
%env TFIO_DEMO_DATABASE_NAME=db
%env TFIO_DEMO_DATABASE_HOST=localhost
%env TFIO_DEMO_DATABASE_PORT=5432
%env TFIO_DEMO_DATABASE_USER=postgres
%env TFIO_DEMO_DATABASE_PASS=postgres

env: TFIO_DEMO_DATABASE_NAME=db
env: TFIO_DEMO_DATABASE_HOST=localhost
env: TFIO_DEMO_DATABASE_PORT=5432
env: TFIO_DEMO_DATABASE_USER=postgres
env: TFIO_DEMO_DATABASE_PASS=postgres


upload init.sql on /content/ 

In [ ]:
!PGPASSWORD=$TFIO_DEMO_DATABASE_PASS psql -q -h $TFIO_DEMO_DATABASE_HOST -p $TFIO_DEMO_DATABASE_PORT -U $TFIO_DEMO_DATABASE_USER -d $TFIO_DEMO_DATABASE_NAME -f init.sql

psql:init.sql:6: ERROR:  relation "star" already exists
psql:init.sql:17: ERROR:  relation "planet" already exists
psql:init.sql:20: ERROR:  duplicate key value violates unique constraint "star_pkey"
DETAIL:  Key (kepler_id)=(2713049) already exists.
psql:init.sql:21: ERROR:  duplicate key value violates unique constraint "star_pkey"
DETAIL:  Key (kepler_id)=(3114167) already exists.
psql:init.sql:22: ERROR:  duplicate key value violates unique constraint "star_pkey"
DETAIL:  Key (kepler_id)=(3115833) already exists.
psql:init.sql:23: ERROR:  duplicate key value violates unique constraint "star_pkey"
DETAIL:  Key (kepler_id)=(3246984) already exists.
psql:init.sql:24: ERROR:  duplicate key value violates unique constraint "star_pkey"
DETAIL:  Key (kepler_id)=(3342970) already exists.
psql:init.sql:25: ERROR:  duplicate key value violates unique constraint "star_pkey"
DETAIL:  Key (kepler_id)=(3351888) already exists.
psql:init.sql:26: ERROR:  duplicate key value violates unique constra

In [ ]:
import psycopg2

# Establish the connection
conn = psycopg2.connect(host="localhost",database="db", user="postgres", password="postgres")
cursor = conn.cursor()

# Execute an SQL query and receive the output
cursor.execute('SELECT 2 + 3;')
records = cursor.fetchall()

print(records)

[(5,)]


We will go through the example on the last slide step by step.

1. Establish a connection to the database

conn = psycopg2.connect(dbname='db', user='grok')
This command initialises a new database session and returns a connection object. We have to specify the name of the database and the name of the user. Note that the dbname is the name of the database, not a table in the database.

Throughout this module we're calling our database 'db'. On your local machine, you would use your user account name for the user. Here we're going to use 'grok'.

2. Create a cursor object

cursor = conn.cursor()
The cursor is the object that interfaces with the database. We can execute SQL queries and receive their output through this object. We can call the object's functions by using the dot (.) notation just like we do for modules. The two functions that we will use most often are execute and fetchall.

3. Run a SQL query

cursor.execute('SELECT 2 + 3;')
To run a SQL query, we call the execute function, which is a function of the cursor object. This function takes the SQL query in form of a string as its argument.

4. Receive the query return

records = cursor.fetchall()
The fetchall function returns the output of the last query. When taking SQL data into Python, the data types are converted to the closest match in Python data types. We'll have a closer look at this later.

Test

In [ ]:
def select_all(table):
  cursor.execute('SELECT * FROM '+table+';')
  records = cursor.fetchall()
  return records
print(select_all('Star'))

[(2713049, 5996, 0.956), (3114167, 5666, 0.677), (3115833, 5995, 0.847), (3246984, 5735, 0.973), (3342970, 6167, 1.064), (3351888, 5717, 1.057), (3453214, 5733, 0.77), (3641726, 5349, 0.82), (3832474, 5485, 0.867), (3935914, 5934, 0.893), (3940418, 5170, 0.807), (4049131, 4905, 0.761), (4139816, 3887, 0.48), (4275191, 5557, 0.781), (4476123, 5413, 0.751), (5358241, 6079, 0.945), (5358624, 5071, 0.788), (5456651, 4980, 0.734), (6862328, 5796, 0.871), (6922244, 6225, 1.451), (8395660, 5881, 1.029), (9579641, 6391, 1.332), (10187017, 4812, 0.755), (10480982, 6117, 0.947), (10526549, 4856, 0.696), (10583066, 4536, 0.693), (10601284, 5559, 0.806), (10662202, 4722, 0.527), (10666592, 6350, 1.991), (10682541, 5339, 0.847), (10797460, 5850, 1.04), (10811496, 5853, 0.868), (10848459, 5795, 0.803), (10854555, 6031, 1.046), (10872983, 6046, 0.972), (10875245, 5851, 1.411), (10910878, 5126, 0.742), (10984090, 5803, 1.073), (10987985, 5015, 0.826), (11018648, 5588, 0.796), (11138155, 6117, 1.025), 

In [ ]:
def column_stats(table,col):
  
  import numpy as np
  cursor.execute('SELECT '+col+' FROM '+table+';')
  records = np.array(cursor.fetchall())
  return (np.mean(records),np.median(records))
print(column_stats('Star','t_eff'))

(5490.681818181818, 5634.0)
